In [1]:
import urllib.parse
from pathlib import Path

import psycopg
from psycopg.rows import dict_row
from tqdm.auto import tqdm

from fishsense_data_processing_spider.backend import get_project_export
from fishsense_data_processing_spider.config import PG_CONN_STR, settings

In [2]:
export = get_project_export(
    project_id=19,
    label_studio_api_key=settings.label_studio.api_key,
    label_studio_host=settings.label_studio.host
)

In [27]:
cksums = {task['id']:Path(urllib.parse.urlparse(task['data']['img']).path).stem for task in export}
results = {task['id']: task['annotations'][0]['result']
           for task in export
           if len(task['annotations']) > 0 and len(task['annotations'][0]['result']) > 0}

In [32]:
flat_results = {
    cksums[task_id]:{
        f'{result[0]['value']['keypointlabels'][0]}_x': int(result[0]['value']['x'] / 100 * result[0]['original_width']),
        f'{result[0]['value']['keypointlabels'][0]}_y': int(result[0]['value']['y'] / 100 * result[0]['original_height']),
        f'{result[1]['value']['keypointlabels'][0]}_x': int(result[1]['value']['x'] / 100 * result[1]['original_width']),
        f'{result[1]['value']['keypointlabels'][0]}_y': int(result[1]['value']['y'] / 100 * result[1]['original_height']),
    }
    for task_id, result in results.items()
    if len(result) == 2
}

In [36]:
params_seq = [
    {
        'cksum': cksum,
        'head_x': coords['Snout_x'],
        'head_y': coords['Snout_y'],
        'tail_x': coords['Fork_x'],
        'tail_y': coords['Fork_y']
    }
    for cksum, coords in flat_results.items()
    if 'Snout_x' in coords
]

In [37]:
with psycopg.connect(PG_CONN_STR, row_factory=dict_row) as con, con.cursor() as cur:
    cur.executemany(
        '''
INSERT INTO headtail_labels (cksum, head_x, head_y, tail_x, tail_y)
VALUES (%(cksum)s, %(head_x)s, %(head_y)s, %(tail_x)s, %(tail_y)s)
ON CONFLICT DO NOTHING;
        ''',
        params_seq
    )
    con.commit()